# Collector
## Overview
From its literal meaning, we can easily know that the Collector in Tianshou is used to collect training data. More specifically, the Collector controls the interaction between Policy (agent) and the environment. It also helps save the interaction data into the ReplayBuffer and returns episode statistics.

## Usages
### Policy evaluation
We need to evaluate our trained policy from time to time in DRL experiments. Collector can help us with this.

First we have to initialise a Collector with an (vectorised) environment and a gieven policy (agent).

In [2]:
import gymnasium as gym
import numpy as np
import torch

from tianshou.data import Collector
from tianshou.env import DummyVectorEnv
from tianshou.policy import PGPolicy
from tianshou.utils.net.common import Net
from tianshou.utils.net.discrete import Actor

import warnings
warnings.filterwarnings("ignore")

env = gym.make("CartPole-v0")
test_envs = DummyVectorEnv([lambda: gym.make("CartPole-v0") for _ in range(2)])

# model
net = Net(env.observation_space.shape, hidden_sizes=[16,])
actor = Actor(net, env.action_space.shape)
optim = torch.optim.Adam(actor.parameters(), lr=0.0003)

policy = PGPolicy(actor, optim, dist_fn=torch.distributions.Categorical)
test_collector = Collector(policy, test_envs)

Now we would like to collect 9 episodes of data to test how our initialised Policy performs.

In [3]:
collect_result = test_collector.collect(n_episode=9)
print(collect_result)
print("Rewards of 9 episodes are {}".format(collect_result["rews"]))
print("Avergae episode reward is {}".format(collect_result["rew"]))
print("Average episode length is {}".format(collect_result["len"]))

{'n/ep': 9, 'n/st': 85, 'rews': array([ 9., 10.,  9., 10., 10.,  9.,  8., 10., 10.]), 'lens': array([ 9, 10,  9, 10, 10,  9,  8, 10, 10]), 'idxs': array([0, 1, 0, 1, 0, 1, 1, 0, 1]), 'rew': 9.444444444444445, 'len': 9.444444444444445, 'rew_std': 0.6849348892187751, 'len_std': 0.6849348892187751}
Rewards of 9 episodes are [ 9. 10.  9. 10. 10.  9.  8. 10. 10.]
Avergae episode reward is 9.444444444444445
Average episode length is 9.444444444444445


Now we wonder what is the performance of a random policy.

In [4]:
# reset the collector
test_collector.reset()
collect_result = test_collector.collect(n_episode=9, random=True)
print(collect_result)
print("Rewards of 9 episodes are {}".format(collect_result["rews"]))
print("Avergae episode reward is {}".format(collect_result["rew"]))
print("Average episode length is {}".format(collect_result["len"]))

{'n/ep': 9, 'n/st': 240, 'rews': array([19., 52., 15., 16., 64., 16., 22., 10., 26.]), 'lens': array([19, 52, 15, 16, 64, 16, 22, 10, 26]), 'idxs': array([1, 0, 0, 0, 1, 1, 0, 0, 1]), 'rew': 26.666666666666668, 'len': 26.666666666666668, 'rew_std': 17.505554674001175, 'len_std': 17.505554674001175}
Rewards of 9 episodes are [19. 52. 15. 16. 64. 16. 22. 10. 26.]
Avergae episode reward is 26.666666666666668
Average episode length is 26.666666666666668


Seems that an initialised policy performs even worse than a random policy without any training.

### Data Collecting
Data collecting is mostly used during training, when we need to store the collected data in a ReplayBuffer.

In [5]:
from tianshou.data import VectorReplayBuffer
train_env_num = 4
buffer_size = 100
train_envs = DummyVectorEnv([lambda: gym.make("CartPole-v0") for _ in range(train_env_num)])
replaybuffer = VectorReplayBuffer(buffer_size, train_env_num)

train_collector = Collector(policy, train_envs, replaybuffer)

Now we can collect 50 steps of data, which will be automatically saved in the replay buffer. You can still choose to collect a certain number of episodes rather than steps.

In [6]:
print(len(replaybuffer))
collect_result = train_collector.collect(n_step=50)
print(len(replaybuffer))
print(collect_result)

0
52
{'n/ep': 4, 'n/st': 52, 'rews': array([ 9.,  9., 10., 10.]), 'lens': array([ 9,  9, 10, 10]), 'idxs': array([25, 75,  0, 50]), 'rew': 9.5, 'len': 9.5, 'rew_std': 0.5, 'len_std': 0.5}


In [7]:
for i in range(13):
    print(i, replaybuffer.next(i))

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 9
10 11
11 12
12 12


In [8]:
replaybuffer.sample(10)

(Batch(
     obs: array([[-1.30994722e-01, -1.58440459e+00,  1.12622060e-01,
                   2.28949118e+00],
                 [-1.03222884e-01, -1.38859224e+00,  7.31204748e-02,
                   1.97507930e+00],
                 [-2.84368210e-02, -5.57652116e-01,  4.86735404e-02,
                   8.95394683e-01],
                 [-4.29266058e-02, -3.74700904e-01,  5.52442893e-02,
                   6.50565684e-01],
                 [-4.97634150e-02, -1.19769776e+00,  5.74298427e-02,
                   1.76256251e+00],
                 [ 1.20251626e-02, -2.80111413e-02, -3.13451886e-02,
                   2.80445516e-02],
                 [-1.33525860e-03, -6.12109601e-01, -1.27009312e-02,
                   8.78336012e-01],
                 [-2.97185834e-02, -1.00224161e+00,  2.82057710e-02,
                   1.46120358e+00],
                 [-1.49752086e-04,  7.96206202e-03,  1.99923012e-02,
                  -1.91794746e-02],
                 [ 9.48915476e-06, -1.87440813e